# Import and Install Dependencies

In [4]:
!pip install tensorflow tensorflow-gpu opencv-python sklearn mediapipe matplotlib pandas

^C


You should consider upgrading via the 'c:\users\artem-laptop\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pandas as pd

# Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Model
mp_drawing = mp.solutions.drawing_utils # Drawing

In [3]:
def mediapipe_detections(image, model):
    # Color conversion
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Model stuff
    results = model.process(image)
    
    # Color conversion back
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [21]:
cap = cv2.VideoCapture(1)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read();
        
        if frame is None:
            break
        # Make detections
        image, results = mediapipe_detections(frame, holistic)
        draw_landmarks(image, results)
        
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Extract Keypoint Values

In [5]:
def extract_keypoints(results):
    start_keypoint = results.pose_landmarks.landmark[0] if results.pose_landmarks else 0
    pose = np.array([[res.x - start_keypoint.x, res.y - start_keypoint.y, res.z - start_keypoint.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

    face = np.array([[res.x - start_keypoint.x, res.y - start_keypoint.y, res.z - start_keypoint.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

    lh = np.array([[res.x - start_keypoint.x, res.y - start_keypoint.y, res.z - start_keypoint.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

    rh = np.array([[res.x - start_keypoint.x, res.y - start_keypoint.y, res.z - start_keypoint.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])

In [15]:
extract_keypoints(results)

array([0.        , 0.        , 0.        , ..., 0.03640687, 0.18705031,
       0.53496515])

# Setup folders structure

In [6]:
DATA_PATH = os.path.join('data/csv')
actions = np.array([name for name in os.listdir("data/csv")])

# collect 30 videos for any action
no_sequences = 30

# videos are going to be 30 frames in length
sequence_length = 30

In [7]:
for action in actions:
    try:
        os.makedirs(os.path.join(DATA_PATH, action))
    except:
        pass

# Collect Keypoints Values for Training and Testing

In [8]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
     # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        
        videos = np.array([name for name in os.listdir("data/raw/" + action)])
        
        for video in videos:
            cap = cv2.VideoCapture("./data/raw/" + action + "/" + video)
            
            keypoints = []
            
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                ret, frame = cap.read();
                
                if frame is None:
                    break

                # Make detections
                image, results = mediapipe_detections(frame, holistic)
                keypoints.append(extract_keypoints(results))
                
                    
            df = pd.DataFrame(keypoints)
            path = os.path.join(DATA_PATH, action, str(video) + '.csv')
            df.to_csv(path)        
        
            cap.release()
            cv2.destroyAllWindows()

In [16]:
import time

time.sleep(5)

cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in ['hi', 'goodbye', 'gluhoi', 'how']:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            keypoints = []
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detections(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints.append(extract_keypoints(results))
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            df = pd.DataFrame(keypoints)
            path = os.path.join(DATA_PATH, action, str(sequence) + str(round(time.time())) + '.csv')
            df.to_csv(path)         
    cap.release()
    cv2.destroyAllWindows()

# Preprocess Data and Create Labels and Features

In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [19]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'gluhoi': 0,
 'goodbye': 1,
 'hello': 2,
 'hi': 3,
 'how': 4,
 'I': 5,
 'thanks': 6,
 'things': 7,
 'you': 8}

In [20]:
sequences, labels = [], []
for action in actions:
    csvs = np.array([name for name in os.listdir("data/csv/" + action)])    
    for csv in csvs:
        df = pd.read_csv("data/csv/" + action + "/" + csv)
        df = df.drop(['0'], axis = 1)
        
        sequences.append(df.values.tolist())
        labels.append(label_map[action])

In [21]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Build and Train LSTM Neural Network

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Dropout, GRU
from tensorflow.keras.callbacks import TensorBoard

In [23]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [29]:
# Build a model
model = Sequential()
model.add(Conv1D(
    filters=64,
    input_shape=(30,1662),
    kernel_size=2,
    activation="relu"))
model.add(MaxPooling1D(2))
model.add(Conv1D(
    filters=128,
    kernel_size=2,
    activation="relu"))
model.add(MaxPooling1D(2))


model.add(GRU(64, return_sequences=True, activation='tanh'))
model.add(GRU(128, return_sequences=True, activation='tanh'))
model.add(GRU(64, return_sequences=False, activation='tanh'))

model.add(Dense(64, activation='relu'))

model.add(Dense(32, activation='relu'))



model.add(Dense(actions.shape[0], activation='softmax'))

In [25]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
22/22 [==============================] - 12s 104ms/step - loss: 2.1334 - categorical_accuracy: 0.1479
Epoch 2/1000
22/22 [==============================] - 1s 41ms/step - loss: 1.8276 - categorical_accuracy: 0.3338
Epoch 3/1000
22/22 [==============================] - 1s 28ms/step - loss: 1.4129 - categorical_accuracy: 0.4261
Epoch 4/1000
22/22 [==============================] - 0s 16ms/step - loss: 1.1815 - categorical_accuracy: 0.4993
Epoch 5/1000
22/22 [==============================] - 0s 18ms/step - loss: 0.8939 - categorical_accuracy: 0.5988
Epoch 6/1000
22/22 [==============================] - 0s 18ms/step - loss: 0.7598 - categorical_accuracy: 0.6735
Epoch 7/1000
22/22 [==============================] - 0s 22ms/step - loss: 0.5863 - categorical_accuracy: 0.7379
Epoch 8/1000
22/22 [==============================] - 0s 22ms/step - loss: 0.5204 - categorical_accuracy: 0.7438
Epoch 9/1000
22/22 [==============================] - 1s 31ms/step - loss: 0.5805 - categorica

22/22 [==============================] - 0s 18ms/step - loss: 0.0815 - categorical_accuracy: 0.9722
Epoch 73/1000
22/22 [==============================] - 0s 20ms/step - loss: 0.1139 - categorical_accuracy: 0.9707
Epoch 74/1000
22/22 [==============================] - 1s 24ms/step - loss: 0.0644 - categorical_accuracy: 0.9795
Epoch 75/1000
22/22 [==============================] - 1s 30ms/step - loss: 0.1271 - categorical_accuracy: 0.9575
Epoch 76/1000
22/22 [==============================] - 0s 16ms/step - loss: 0.0817 - categorical_accuracy: 0.9707
Epoch 77/1000
22/22 [==============================] - 0s 17ms/step - loss: 0.0933 - categorical_accuracy: 0.9736
Epoch 78/1000
22/22 [==============================] - 0s 19ms/step - loss: 0.1879 - categorical_accuracy: 0.9429
Epoch 79/1000
22/22 [==============================] - 0s 18ms/step - loss: 0.1216 - categorical_accuracy: 0.9619
Epoch 80/1000
22/22 [==============================] - 1s 35ms/step - loss: 0.1077 - categorical_accur

22/22 [==============================] - 0s 20ms/step - loss: 0.0498 - categorical_accuracy: 0.9854
Epoch 143/1000
22/22 [==============================] - 0s 18ms/step - loss: 0.0308 - categorical_accuracy: 0.9927
Epoch 144/1000
22/22 [==============================] - 0s 17ms/step - loss: 0.0158 - categorical_accuracy: 0.9971
Epoch 145/1000
22/22 [==============================] - 0s 17ms/step - loss: 0.0176 - categorical_accuracy: 0.9971
Epoch 146/1000
22/22 [==============================] - 0s 19ms/step - loss: 0.0188 - categorical_accuracy: 0.9956
Epoch 147/1000
22/22 [==============================] - 0s 20ms/step - loss: 0.0136 - categorical_accuracy: 0.9956
Epoch 148/1000
22/22 [==============================] - 0s 17ms/step - loss: 0.0113 - categorical_accuracy: 0.9971
Epoch 149/1000
22/22 [==============================] - 0s 16ms/step - loss: 0.0084 - categorical_accuracy: 0.9985
Epoch 150/1000
22/22 [==============================] - 0s 17ms/step - loss: 0.0131 - categoric

22/22 [==============================] - 0s 11ms/step - loss: 0.0096 - categorical_accuracy: 0.9985
Epoch 214/1000
22/22 [==============================] - 0s 12ms/step - loss: 0.0212 - categorical_accuracy: 0.9971
Epoch 215/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.0730 - categorical_accuracy: 0.9795
Epoch 216/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.1092 - categorical_accuracy: 0.9517
Epoch 217/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.0261 - categorical_accuracy: 0.9956
Epoch 218/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.0103 - categorical_accuracy: 0.9971
Epoch 219/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.0086 - categorical_accuracy: 0.9985
Epoch 220/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.0161 - categorical_accuracy: 0.9971
Epoch 221/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.0770 - categoric

22/22 [==============================] - 0s 11ms/step - loss: 0.1952 - categorical_accuracy: 0.9502
Epoch 285/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.2030 - categorical_accuracy: 0.9444
Epoch 286/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.1321 - categorical_accuracy: 0.9649
Epoch 287/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.0393 - categorical_accuracy: 0.9883
Epoch 288/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.0213 - categorical_accuracy: 0.9941
Epoch 289/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.0146 - categorical_accuracy: 0.9971
Epoch 290/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.0197 - categorical_accuracy: 0.9941
Epoch 291/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.0193 - categorical_accuracy: 0.9971
Epoch 292/1000
22/22 [==============================] - 0s 11ms/step - loss: 0.0212 - categoric

...

22/22 [==============================] - 0s 11ms/step - loss: 1.1498e-06 - categorical_accuracy: 1.0000
Epoch 697/1000
22/22 [==============================] - 0s 11ms/step - loss: 1.1383e-06 - categorical_accuracy: 1.0000
Epoch 698/1000
22/22 [==============================] - 0s 11ms/step - loss: 1.1214e-06 - categorical_accuracy: 1.0000
Epoch 699/1000
22/22 [==============================] - 0s 10ms/step - loss: 1.1078e-06 - categorical_accuracy: 1.0000
Epoch 700/1000
22/22 [==============================] - 0s 11ms/step - loss: 1.0931e-06 - categorical_accuracy: 1.0000
Epoch 701/1000
22/22 [==============================] - 0s 11ms/step - loss: 1.0807e-06 - categorical_accuracy: 1.0000
Epoch 702/1000
22/22 [==============================] - 0s 11ms/step - loss: 1.0664e-06 - categorical_accuracy: 1.0000
Epoch 703/1000
22/22 [==============================] - 0s 11ms/step - loss: 1.0539e-06 - categorical_accuracy: 1.0000
Epoch 704/1000
22/22 [==============================] - 0s 11ms

Epoch 765/1000
22/22 [==============================] - 0s 11ms/step - loss: 4.8085e-07 - categorical_accuracy: 1.0000
Epoch 766/1000
22/22 [==============================] - 0s 11ms/step - loss: 4.7684e-07 - categorical_accuracy: 1.0000
Epoch 767/1000
22/22 [==============================] - 0s 11ms/step - loss: 4.6916e-07 - categorical_accuracy: 1.0000
Epoch 768/1000
22/22 [==============================] - 0s 10ms/step - loss: 4.6200e-07 - categorical_accuracy: 1.0000
Epoch 769/1000
22/22 [==============================] - 0s 11ms/step - loss: 4.5833e-07 - categorical_accuracy: 1.0000
Epoch 770/1000
22/22 [==============================] - 0s 10ms/step - loss: 4.5188e-07 - categorical_accuracy: 1.0000
Epoch 771/1000
22/22 [==============================] - 0s 11ms/step - loss: 4.4699e-07 - categorical_accuracy: 1.0000
Epoch 772/1000
22/22 [==============================] - 0s 11ms/step - loss: 4.4018e-07 - categorical_accuracy: 1.0000
Epoch 773/1000
22/22 [==========================

22/22 [==============================] - 0s 10ms/step - loss: 2.0683e-07 - categorical_accuracy: 1.0000
Epoch 834/1000
22/22 [==============================] - 0s 10ms/step - loss: 2.0438e-07 - categorical_accuracy: 1.0000
Epoch 835/1000
22/22 [==============================] - 0s 10ms/step - loss: 2.0211e-07 - categorical_accuracy: 1.0000
Epoch 836/1000
22/22 [==============================] - 0s 10ms/step - loss: 1.9950e-07 - categorical_accuracy: 1.0000
Epoch 837/1000
22/22 [==============================] - 0s 10ms/step - loss: 1.9862e-07 - categorical_accuracy: 1.0000
Epoch 838/1000
22/22 [==============================] - 0s 10ms/step - loss: 1.9566e-07 - categorical_accuracy: 1.0000
Epoch 839/1000
22/22 [==============================] - 0s 11ms/step - loss: 1.9304e-07 - categorical_accuracy: 1.0000
Epoch 840/1000
22/22 [==============================] - 0s 10ms/step - loss: 1.9112e-07 - categorical_accuracy: 1.0000
Epoch 841/1000
22/22 [==============================] - 0s 10ms

Epoch 902/1000
22/22 [==============================] - 0s 11ms/step - loss: 7.6098e-08 - categorical_accuracy: 1.0000
Epoch 903/1000
22/22 [==============================] - 0s 11ms/step - loss: 7.4702e-08 - categorical_accuracy: 1.0000
Epoch 904/1000
22/22 [==============================] - 0s 10ms/step - loss: 7.3655e-08 - categorical_accuracy: 1.0000
Epoch 905/1000
22/22 [==============================] - 0s 10ms/step - loss: 7.2608e-08 - categorical_accuracy: 1.0000
Epoch 906/1000
22/22 [==============================] - 0s 10ms/step - loss: 7.1910e-08 - categorical_accuracy: 1.0000
Epoch 907/1000
22/22 [==============================] - 0s 11ms/step - loss: 7.1211e-08 - categorical_accuracy: 1.0000
Epoch 908/1000
22/22 [==============================] - 0s 11ms/step - loss: 7.0688e-08 - categorical_accuracy: 1.0000
Epoch 909/1000
22/22 [==============================] - 0s 11ms/step - loss: 6.9641e-08 - categorical_accuracy: 1.0000
Epoch 910/1000
22/22 [==========================

22/22 [==============================] - 0s 11ms/step - loss: 2.9846e-08 - categorical_accuracy: 1.0000
Epoch 971/1000
22/22 [==============================] - 0s 10ms/step - loss: 2.9497e-08 - categorical_accuracy: 1.0000
Epoch 972/1000
22/22 [==============================] - 0s 11ms/step - loss: 2.9497e-08 - categorical_accuracy: 1.0000
Epoch 973/1000
22/22 [==============================] - 0s 11ms/step - loss: 2.8973e-08 - categorical_accuracy: 1.0000
Epoch 974/1000
22/22 [==============================] - 0s 11ms/step - loss: 2.8973e-08 - categorical_accuracy: 1.0000
Epoch 975/1000
22/22 [==============================] - 0s 11ms/step - loss: 2.8624e-08 - categorical_accuracy: 1.0000
Epoch 976/1000
22/22 [==============================] - 0s 11ms/step - loss: 2.8101e-08 - categorical_accuracy: 1.0000
Epoch 977/1000
22/22 [==============================] - 0s 11ms/step - loss: 2.7926e-08 - categorical_accuracy: 1.0000
Epoch 978/1000
22/22 [==============================] - 0s 11ms

In [27]:
model.evaluate(X_test, y_test)

3/3 [==============================] - 1s 31ms/step - loss: 0.1247 - categorical_accuracy: 0.9868


[0.12473362684249878, 0.9868420958518982]

# Make Predictions 

In [59]:
res = model.predict(X_test)

In [60]:
actions[np.argmax(res[0])]

'thanks'

In [61]:
actions[np.argmax(y_test[0])]

'thanks'

In [34]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, confusion_matrix

In [62]:
yhat = model.predict(X_test)

ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [63]:
confusion_matrix(ytrue, yhat)

array([[ 8,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  8,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  5,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  3,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 14,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  7,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  8,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  5,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  7]], dtype=int64)

# Test in Real Time 

In [28]:
model.save('action.h5')
del model

In [30]:
model.load_weights('action.h5')

In [31]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    
        
        
    return output_frame

In [33]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.75

cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

    # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detections(frame, holistic)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-5:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
how
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
hi
hi
hi
you
you
you
how
how
how
how
how
you
how
how
how
how
how
how
how
how
how
I
I
I
I
I
I
I
I
I
how
how
how
how
I
I
I
I
I
I
I
you
you
you
you
you
you
you
you
you
I
I
I
I
I
I
I
I
I
I
gluhoi
gluhoi
gluhoi
gluhoi
gluhoi
gluhoi
gluhoi
gluhoi
gluhoi
gluhoi
gluhoi
gluhoi
gluhoi
gluhoi
gluhoi
gluhoi
hi
hi
goodbye
goodbye
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
hi
gluhoi
gluhoi
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
how
how
how
h

In [16]:
threshold

0.7